In [1]:
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredPDFLoader
from REMOVED_SECRET import Document as LangchainDocument
from langchain_community.vectorstores import FAISS
from REMOVED_SECRET import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import torch
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, TextStreamer
from ragatouille import RAGPretrainedModel
import nltk
from transformers import AutoTokenizer, AutoModelForCausalLM
from rag_for_notebook_sunday import CustomTextGenerationPipeline
import REMOVED_SECRET as F

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.REMOVED_SECRET4
  warnings.warn(
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `REMOVED_SECRET.GradScaler(args...)` is deprecated. Please use `REMOVED_SECRET('cuda', args...)` instead.
  self.scaler = REMOVED_SECRET.GradScaler()


=> Retrieving documents...


Process SpawnProcess-2:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/obb/codes/langers/.venv/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py", line 514, in _encode_multi_process_worker
    embeddings = model.encode(
  File "/home/obb/codes/langers/.venv/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py", line 337, in encode
    self.to(device)
  File "/home/obb/codes/langers/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1174, in to
    return self._apply(convert)
  File "/home/obb/codes/langers/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 780, in _apply
    module._apply(fn)
  File "/home/obb/codes/langers/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 

KeyboardInterrupt: 

In [2]:
custom_pipeline = CustomTextGenerationPipeline(model_id="microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [3]:
prompt = "Hello, how are you?"

generated_sequence, all_logits = custom_pipeline(prompt)


#check if logits require gradients
print(f"Logits requires_grad: {all_logits.requires_grad}")



/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/home/obb/codes/langers/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


I've been considering the benefits and risks of taking my dog
Logits requires_grad: True


In [5]:
import torch
import REMOVED_SECRET as F
print(f"all_logits shape: {all_logits.shape}")
# Assuming all_logits is your logits tensor
#batch_size, sequence_length, vocab_size = all_logits.size()
sequence_length, vocab_size = all_logits.size()
# Example target tensor: must be of shape (batch_size * sequence_length)
# Replace this with your actual target IDs. Here, we're simulating with random targets
target = torch.randint(0, vocab_size, (sequence_length,)).to(all_logits.device)
# Flatten the logits for loss computation
#logits_for_loss = all_logits.view(-1, vocab_size)  # shape: (batch_size * sequence_length, vocab_size)
#print(f"logits_for_loss shape: {logits_for_loss.shape}")
print(f"target shape: {target.shape}")
# Calculate the cross-entropy loss
loss = F.cross_entropy(all_logits, target)

# Backpropagation to compute gradients
loss.backward()

print(f"Gradients computed for logits: {all_logits.grad is not None}")
print("Grads:", all_logits.grad)

all_logits shape: torch.Size([1, 448896])
target shape: torch.Size([1])
Gradients computed for logits: True
